1. https://www.bilibili.com/video/BV18g4119737/?p=53&vd_source=70200f7d09862fd682e5f89b22c89125
2. visdom可视化的问题

# 0. Preparations

In [ ]:
import torch
import numpy as np
from tensorboardX import SummaryWriter
from visdom import Visdom

# 1. tensorboard usage
1. 新建实例
2. 主要用`add_scalar`添加要监听的数据
3. Add image and text data to summary.
4. 也可以画直方图

In [ ]:
# 新建实例
writer = SummaryWriter()
# 给要监听的数据dummy_s1[0]起别名'data/scalar1',n_iter表示epoch,是x轴坐标,哪一个时间戳的数据
writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
# 监听多个数据。n_iter表示epoch,是x轴坐标,对应多个y轴数据
writer.add_scalar('data/scalar_group',{'xsinx':n_iter*np.sin(n_iter),
                                       'xcosx':n_iter*np.cos(n_iter),
                                       'arctanx':np.arctan(n_iter)}    ,n_iter)
# Add image data to summary.
writer.add_image('Image',x,x_iter)
writer.add_text('Text','text logged at step:'+str(n_iter),n_iter)

# histogram
writer.add_histogram(name, param.clone().cpu().data.numpy(),n_iter)

5. 要注意tensorboard实际上**抽取的是numpy数据**,往往需要上面,`param.clone().cpu().data.numpy()`转到cpu上再转到numpy

# 2. visdom usage
1. visdom可视化是facebook研究的,可以原生的接受tensor数据,实际上帮你做了`.data.numpy()`这一步。
2. visdom比tensorboard效率更高,因为tensorboard会写到文件里去占用大量空间
3. tensorboard每30s更新一次（不算高）,而且visdom窗口更好看

## 2.1 下载与开启visdom
1. 下载直接`pip`. 开启visdom本质是开启web服务器:`python -m visdom.server`
2. https://blog.csdn.net/qq_43855428/article/details/127302873

## 2.2 basic usage: single trace
1. 创建实例,然后创建直线。`win='train_loss'`是唯一的标志符
2. visdom管理窗口的办法是：
    - 一个大的屏幕叫做environment,对应一个大的工程。不指定的话用的就是这个大的env
    - 一个环境里可以有很多小windows,`win='train_loss`就在大env里创建小的windows
    -
3. 注意image可以接收tensor类型,但是这里的line还是接收一个numpy数据。

In [ ]:
# 创建实例
viz = Visdom()
# 创建直线,然后添加数据到直线上,前两个数表示Y和X。win='train_loss'是唯一的标志符
viz.line([0.],[0.], win='train_loss',opts = dict(title='train loss'))
# 不断的把数据添加到直线上: update='append'表示接续上次更新（不覆盖）
viz.line([loss.item()],[global_step], win='train_loss', update='append')


## 2.3 basic usage: multi-traces
1. 一个window里画多条曲线时,legend设定了两个直线的label

In [ ]:
viz = Visdom()
# 多条曲线时,y的初始值变了.legend设定了两个直线的label
viz.line([0.0,0.0],[0.], win='test',opts = dict(title='test loss&acc',
                                                legend=['loss', 'acc']),)
# 注意list的括号[loss.item(), acc.item()]外面还有一个
viz.line([[loss.item(), acc.item()]],[global_step], win='test', update='append')

## 2.4 basic usage: visual X
1. 如果要看一张image,data是一个`[b,1,28,28]`的图片。
    - 对于tensorboard 要转换成numpy。visdom直接用tensor就可以看
    - `data`是一个tensor

In [ ]:
viz = Visdom()
viz.images(data.view(-1,1,28,28), win='x')

